In [2]:
import soothsayer as sy
%run ~/Google/.ipython-profile.py

1.266822338104248 seconds


In [3]:
df_exons = sy.io.read_gff3("./grch38/Homo_sapiens.GRCh38.94.gff3.gz", record_type="exon", verbose=True)

Reading gff3 file:	./grch38/Homo_sapiens.GRCh38.94.gff3.gz
Processing lines: 100%|██████████| 2814993/2814993 [00:05<00:00, 494210.35it/s]
Splitting fields: 1262162it [04:16, 4924.09it/s]


In [5]:
%%time
chromosomes = list()
for file in os.scandir("./grch38/chromosomes/"):
    if file.name.endswith(".gz"):
        chromosomes.append(sy.read_fasta(file, verbose=True, description=False))
Se_assembly = pd.concat(chromosomes)


Reading sequence file: ./grch38/chromosomes/Homo_sapiens.GRCh38.dna.chromosome.1.fa.gz: 1it [00:06,  6.36s/it]
Reading sequence file: ./grch38/chromosomes/Homo_sapiens.GRCh38.dna.chromosome.10.fa.gz: 1it [00:03,  3.39s/it]
Reading sequence file: ./grch38/chromosomes/Homo_sapiens.GRCh38.dna.chromosome.11.fa.gz: 1it [00:03,  3.48s/it]
Reading sequence file: ./grch38/chromosomes/Homo_sapiens.GRCh38.dna.chromosome.12.fa.gz: 1it [00:03,  3.35s/it]
Reading sequence file: ./grch38/chromosomes/Homo_sapiens.GRCh38.dna.chromosome.13.fa.gz: 1it [00:02,  2.81s/it]
Reading sequence file: ./grch38/chromosomes/Homo_sapiens.GRCh38.dna.chromosome.14.fa.gz: 1it [00:02,  2.62s/it]
Reading sequence file: ./grch38/chromosomes/Homo_sapiens.GRCh38.dna.chromosome.15.fa.gz: 1it [00:02,  2.47s/it]
Reading sequence file: ./grch38/chromosomes/Homo_sapiens.GRCh38.dna.chromosome.16.fa.gz: 1it [00:02,  2.21s/it]
Reading sequence file: ./grch38/chromosomes/Homo_sapiens.GRCh38.dna.chromosome.17.fa.gz: 1it [00:02,  2.0

CPU times: user 1min 17s, sys: 4.29 s, total: 1min 21s
Wall time: 1min 24s


In [182]:
import soothsayer as sy
from soothsayer.utils import format_path, infer_compression
def infer_compression(path:str):
    path = format_path(path)
    compression = None
    ext_zip = (".zip")
    ext_gzip = (".gz", ".gzip", ".pgz")
    ext_bz2 = (".bz2", ".bzip2", ".pbz2")
    if path.endswith(ext_gzip):
        compression= "gzip"
    if path.endswith(ext_bz2):
        compression = "bz2"
    if path.endswith(ext_zip):
        compression = "zip"
    return compression

def get_file_object(path:str, mode="infer", compression="infer", safe_mode=False, verbose=True):
    """
    with get_file_object("./test.txt.zip", mode="infer", verbose=False) as f_read:
    with get_file_object("./test_write.txt.bz2", mode="write", verbose=False) as f_write:
        for line in f_read.readlines():
            line = str(line.strip())
            print(line, file=f_write)
    """
    # Init
    f = None
    write_file_object = False
    
    # Paths
    path = format_path(path)
    path_exists = os.path.exists(path)

    if verbose:
        header = " ".join(["Processing file:", path])
        print("="*len(header), header, "="*len(header), sep="\n", file=sys.stderr)
    if compression == "infer":
        compression = infer_compression(path)
        if verbose:
            print("Inferring compression:", compression, file=sys.stderr)

    # Inferring mode
    if mode == "infer": # Create new function for this? infer_filemode? 
        if path_exists:
            mode = "read"
        else:
            mode = "write"
    assert mode != "infer", "The mode should be inferred by this point.  Please specify mode manually."
    assert compression != "infer", "The compression should be inferred by this point.  Please specify compression manually."

    # Generic read write
    if mode in ["write", "read"]:
        if mode == "write":
            mode = "w"
        if mode == "read":
            mode = "r"
        if compression in ["gzip", "bz2"]:
            mode = mode + "b"
        if verbose:
            print("Inferring mode:", mode, file=sys.stderr)
            
    # Will a file be written?
    if "w" in mode:
        write_file_object = True
        
    # Ensure zip is not being written
    if write_file_object:
        assert compression != "zip", "Currently cannot handle writing zip files.  Please use gzip, bz2, or None."
        # Future do this:
        # https://docs.python.org/3/library/zipfile.html#zipfile.ZipFile.open
        
    # Safe mode
    if safe_mode:
        if all([write_file_object, path_exists]):
            raise Exception(f"Safe Mode: Please explicitly provide a writeable mode because `{path}` already exists and will be rewritten.")

    # GZIP compression
    if compression == "gzip":
        f = gzip.open(path, mode)
    # BZ2 compression
    if compression == "bz2":
        f = bz2.open(path, mode)
    if compression == "zip":
        filename, ext = os.path.splitext(os.path.split(path)[-1])
        f = zipfile.ZipFile(path,mode).open(filename)
    # No compression
    if f is None:
        return open(path, mode)
    
    # Reading and writing compressed files
    else:
        return TextIOWrapper(f, encoding="utf-8")

    
with get_file_object("/Users/jespinoz/test.txt.zip", mode="infer", verbose=False) as f_read:
    with get_file_object("/Users/jespinoz/test_write.txt.bz2", mode="write", verbose=False) as f_write:
        for line in f_read.readlines():
            line = str(line.strip())
            print(line, file=f_write)

In [149]:
from soothsayer.utils import format_path, infer_compression#, get_file_object

def read_gff3(path:str, compression="infer", record_type=None, verbose = True):
    path = format_path(path)
    if verbose: 
        print(f"Reading gff3 file:",path,sep="\t", file=sys.stderr)
    accepted_recordtypes = {"exon", "gene", "transcript", "protein", None}
    assert record_type in accepted_recordtypes, f"Unrecognized record_type.  Please choose from the following: {accepted_recordtypes}"
    
    # Read the gff3 file
    with get_file_object(path, mode="read", compression=compression, safe_mode=False, verbose=False) as f:
        if verbose:
            iterable_lines = tqdm(f.readlines(), "Processing lines")
        else:
            iterable_lines = f.readlines()
        data= list()
        if record_type is None:
            for line in iterable_lines:
                if not line.startswith("#"):
                    line = line.strip("\n")
                    base_fields = line.split("\t")
                    data.append(base_fields)
        else:
            for line in iterable_lines:
                if not line.startswith("#"):
                    if f"{record_type}_id" in line:
                        line = line.strip("\n")
                        base_fields = line.split("\t")
                        data.append(base_fields)
                
    # Generate table
    df_base = pd.DataFrame(data)
    df_base.columns = ["seq_record", "ensembl", "seq_type", "pos_start", "pos_end", ".1", "sense", ".2", "gff_data"]
    # Splitting fields
    if verbose:
        iterable_fields = tqdm(enumerate(df_base.iloc[:,8].map(lambda x:dict([y.split("=") for y in x.split(";")])).as_matrix()), "Splitting fields")
    else:
        iterable_fields = enumerate(df_base.iloc[:,8].map(lambda x:dict([y.split("=") for y in x.split(";")])).as_matrix())
    df_fields = pd.DataFrame(OrderedDict([(i,pd.Series(v)) for i,v in iterable_fields])).T
    df_gff3 = pd.concat([df_base.iloc[:,:7], df_fields], axis=1)
    
     # Contains identifier
    if "ID" in df_gff3.columns:
        df_gff3["seq_id"] = np.nan
        mask_notnull = df_gff3["ID"].notnull()
        df_gff3["seq_id"][mask_notnull] = df_gff3["ID"][mask_notnull].map(lambda id:id.split(":")[1])
    return df_gff3

In [189]:
# Read Fasta File
def read_fasta(path:str, description:bool=True, case="upper", func_header=None, into=pd.Series, compression="infer", name=None, verbose=True):
    """
    Reads in a single fasta file or a directory of fasta files into a dictionary.
    """
    # Get path
    path = format_path(path)
    
    # Assign pathname as name if there isn't one
    if name is None:
        name = path

    # Open file object
    f = get_file_object(path, mode="read", compression=compression, safe_mode=False, verbose=False)
    
    # Read in fasta
    d_id_seq = OrderedDict()

    if verbose:
        seq_records = tqdm(SeqIO.FastaIO.SimpleFastaParser(f), f"Reading sequence file: {path}")
    else:
        seq_records = SeqIO.FastaIO.SimpleFastaParser(f)

    # Verbose but faster
    if case == "lower":
        for header, seq in seq_records:
            seq = seq.lower()
            if not description:
                header = header.split(" ")[0]
            d_id_seq[header] = seq
    if case == "upper":
        for header, seq in seq_records:
            seq = seq.upper()
            if not description:
                header = header.split(" ")[0]
            d_id_seq[header] = seq
    if case is None:
        for header, seq in seq_records:
            if not description:
                header = header.split(" ")[0]
            d_id_seq[header] = seq

    # Close File
    f.close()

    # Transform header
    if func_header is not None:
        d_id_seq = OrderedDict( [(func_header(id),seq) for id, seq in d_id_seq.items()])
    sequences = into(d_id_seq)
    if hasattr(sequences, "name"):
        sequences.name = name
    return sequences

In [196]:
for ext in ["",".gz", ".bz2", ".zip"]:
    print(ext)
    %timeit read_fasta(f"./cds.fa{ext}", verbose=False)
    


522 µs ± 10.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
.gz
822 µs ± 11.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
.bz2
1.75 ms ± 20.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
.zip
867 µs ± 10.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [154]:
df_gff3exons = read_gff3("/Users/jespinoz/Homo_sapiens.GRCh37.87.chromosome.Y.gff3.gz", record_type="exon")
df_gff3genes = read_gff3("/Users/jespinoz/Homo_sapiens.GRCh37.87.chromosome.Y.gff3.gz", record_type="gene")

Reading gff3 file:	/Users/jespinoz/Homo_sapiens.GRCh37.87.chromosome.Y.gff3.gz
Processing lines: 100%|██████████| 7683/7683 [00:00<00:00, 671238.91it/s]
Splitting fields: 3744it [00:00, 4457.76it/s]
Reading gff3 file:	/Users/jespinoz/Homo_sapiens.GRCh37.87.chromosome.Y.gff3.gz
Processing lines: 100%|██████████| 7683/7683 [00:00<00:00, 1129190.47it/s]
Splitting fields: 495it [00:00, 4739.92it/s]


In [171]:
??infer_compression

Signature: infer_compression(path:str)
Docstring: <no docstring>
Source:   
def infer_compression(path:str):
    path = format_path(path)
    compression = None
    ext_zip = (".zip")
    ext_gzip = (".gz", ".gzip", ".pgz")
    ext_bz2 = (".bz2", ".bzip2", ".pbz2")
    if path.endswith(ext_gzip):
        compression= "gzip"
    if path.endswith(ext_bz2):
        compression = "bz2"
    if path.endswith(ext_zip):
        compression = "zip"
    return compression
File:      ~/anaconda/envs/µ_env/lib/python3.6/site-packages/soothsayer/utils/utils.py
Type:      function


In [27]:

# Forward direction is start-1:end
# str(Se_assembly["1 1 dna:chromosome chromosome:GRCh37:1:1:249250621:1 REF"][11869-1:12227])

# Reverse direction is reverse_complement(start-1:end)
# reverse_complement(str(Se_assembly["1 1 dna:chromosome chromosome:GRCh37:1:1:249250621:1 REF"][14363-1:14829]))

# Note for the reverse, the first exon that appears is the last in the sequence


'GCATCTCTGGGAAAGGACCTGGGGCTGGTGAGGGGCCCGGAGGAGCCTTTGCCCGCGTGTCAGACTCCATCCCTCCTCTGCCGCCACCGCAGCAGCCACAGGCAGAGGAGGACGAGGACGACTGGGAATCGTAGGGGGCTCCATGACACCTTCCCCCCCAGACCCAGACTTGGGCCGTTGCTCTGACATGGACACAGCCAGGACAAGCTGCTCAGACCTACTTCCTTGGGAGGGGGTGACGGAACCAGCACTGTGTGGAGACCAGCTTCAAGGAGCGGAAGGCTGGCTTGAGGCCACACAGCTGGGGCGGGGACTTCTGTCTGCCTGTGCTCCATGGGGGGACGGCTCCACCCAGCCTGCGCCACTGTGTTCTTAAGAGGCTTCCAGAGAAAACGGCACACCAATCAATAAAGAACTGAGCAGAAACCAACAGTGTGCTTTTAATAAAGGATCTCTAGCTGTGCAGG'

In [32]:
df_gff

'ATGACTCCTGTGAGGATGCAGCACTCCCTGGCAGGTCAGACCTATGCCGTGCCCTTCATCCAGCCAGACCTGCGGCGAGAGGAGGCCGTCCAGCAGATGGCGGATGCCCTGCAGTACCTGCAGAAGGTCTCTGGAGACATCTTCAGCAGGTGGCAGGTAGAGCAGAGCCGGAGCCAGGTGCAGGCCATTGGAGAGAAGGTCTCCTTGGCCCAGGCCAAGATTGAGAAGATCAAGGGCAGCAAGAAGGCCATCAAGGTGTTCTCCAGTGCCAAGTACCCTGCTCCAGGGCGCCTGCAGGAATATGGCTCCATCTTCACGGGCGCCCAGGACCCTGGCCTGCAGAGACGCCCCCGCCACAGGATCCAGAGCAAGCACCGCCCCCTGGACGAGCGGGCCCTGCAGGAGAAGCTGAAGGACTTTCCTGTGTGCGTGAGCACCAAGCCGGAGCCCGAGGACGATGCAGAAGAGGGACTTGGGGGTCTTCCCAGCAACATCAGCTCTGTCAGCTCCTTGCTGCTCTTCAACACCACCGAGAACCTCCAGGGGAAGAAGTATGTCTTCCTGGACCCCCTGGCTGGTGCTGTAACAAAGACCCATGTGATGCTGGGGGCAGAGACAGAGGAGAAGCTGTTTGATGCCCCCTTGTCCATCAGCAAGAGAGAGCAGCTGGAACAGCAGGTCCCAGAGAACTACTTCTATGTGCCAGACCTGGGCCAGGTGCCTGAGATTGATGTTCCATCCTACCTGCCTGACCTGCCCGGCATTGCCAACGACCTCATGTACATTGCCGACCTGGGCCCCGGCATTGCCCCCTCTGCCCCTGGCACCATTCCAGAACTGCCCACCTTCCACACTGAGGTAGCCGAGCCTCTCAAGGTAGTGAGAGCCACGAGCCAAGGTGGGCACTTGATGTCGGATCTCTTCAACAAGCTGGTCATGAGGCGCAAGGGCATCTCTGGGAAAGGACCTGGGGCTGGTGAGGGGCCCGGAGGAGCCTTT

array([['7', '127588345', '127588565', '+', 'transcript:ENST00000000233',
        'ENSE00001872691'],
       ['7', '127589083', '127589163', '+', 'transcript:ENST00000000233',
        'ENSE00003494180'],
       ['7', '127589485', '127589594', '+', 'transcript:ENST00000000233',
        'ENSE00003504066'],
       ['7', '127590066', '127590137', '+', 'transcript:ENST00000000233',
        'ENSE00003678978'],
       ['7', '127590963', '127591088', '+', 'transcript:ENST00000000233',
        'ENSE00003676786'],
       ['7', '127591213', '127591705', '+', 'transcript:ENST00000000233',
        'ENSE00000882271']], dtype=object)

In [6]:
fields = ['seq_record', 'pos_start', 'pos_end', 'sense', 'Parent',  'exon_id']
df_exons = df_exons.loc[:,fields].sort_values(["seq_record", "pos_start"], ascending=True)

for id_field in ['pos_start', 'pos_end']:
    df_exons[id_field] = df_exons[id_field].astype(int)
d_id_premrna = OrderedDict()
not_work = list()
for id_transcript, _df_exon in tqdm(df_exons.groupby("Parent", axis=0)):
    id_transcript = id_transcript[11:]
    data = _df_exon.values
    num_exons = data.shape[0]
    id_reference = str(data[0][0])
    sense = data[0][3]
    seq_transcript_build = list()
    if id_reference in Se_assembly:
        seq_reference = Se_assembly[id_reference]
        if data.shape[0] > 1:
            for i in range(0, num_exons):
                exon_start = data[i][1]-1 
                exon_end = data[i][2]
                seq_exon = seq_reference[exon_start:exon_end].upper()
                seq_transcript_build.append(seq_exon)
                if i < (num_exons-1):
                    intron_start = exon_end
                    intron_end = data[i+1][1] - 1
                    seq_intron = seq_reference[intron_start:intron_end].lower()
                    seq_transcript_build.append(seq_intron)
        else:
            pos_start = data[0][1]-1
            pos_end = data[0][2]
            seq_exon = seq_reference[pos_start:pos_end]
            seq_transcript_build.append(seq_exon)
        seq_transcript = "".join(seq_transcript_build)
        if sense == "-":
            seq_transcript = reverse_complement(seq_transcript)
        d_id_premrna[id_transcript] = seq_transcript
    else:
        not_work.append(data)

100%|██████████| 206601/206601 [01:39<00:00, 2068.72it/s]


In [8]:
Se_premrna = pd.Series(d_id_premrna)

In [7]:
# Not ready
from soothsayer.utils import is_dict_like
# Need to group proteins, genes, and transcripts together. have exon separate. have intron w/ unique identifiers based on exons. have premrna a custom one.
def get_regions_from_gff3(path_gff3:str, path_sequences:str, record_type="premrna"):
    
    def _get_exons():
        
    def _get_introns():
    
    def _get_premrna():
        
    def _get_transcripts():
        
    def _get_genes():
        
        
    # Get gff3
    if is_type(path_gff3, pd.DataFrame):
        df_gff3 = path_gff3
    else:
        df_gff3 = read_gff3(path, compression=compression, record_type="exon", verbose=False)
    idx_exons = df_gff3["exon_id"].dropna().index
    df_gff3 = df_gff3.loc[idx_exons,:]
    
    # Get sequences
    if is_dict_like(path_sequences):
        Se_sequences = path_sequences
    else:
        Se_sequences = read_fasta(path_sequences, description=False, verbose=False)

    d_id_seq = OrderedDict()
    for id_transcript, df_gff3__exon in tqdm(df_gff3.groupby("Parent", axis=0)):
        id_transcript = id_transcript[11:]
        data = df_gff3__exon.values
        num_exons = data.shape[0]
        id_reference = str(data[0][0])
        sense = data[0][3]
        seq_reference = Se_sequences[id_reference]
        seq_transcript_build = list()
        if data.shape[0] > 1:
            for i in range(0, num_exons):
                exon_start = data[i][1]-1 
                exon_end = data[i][2]
                seq_exon = seq_reference[exon_start:exon_end].upper()
                seq_transcript_build.append(seq_exon)
                if i < (num_exons-1):
                    intron_start = exon_end
                    intron_end = data[i+1][1] - 1
                    seq_intron = seq_reference[intron_start:intron_end].lower()
                    seq_transcript_build.append(seq_intron)
        else:
            pos_start = data[0][1]-1
            pos_end = data[0][2]
            seq_exon = seq_reference[pos_start:pos_end]
            seq_transcript_build.append(seq_exon)
        seq_transcript = "".join(seq_transcript_build)
        if sense == "-":
            seq_transcript = reverse_complement(seq_transcript)
        d_id_seq[id_transcript] = seq_transcript

In [9]:
sy.write_fasta(Se_premrna, "./grch38/Homo_sapiens.GRCh38.94.premrna.fa")

In [10]:
Se_cds = sy.read_fasta("./grch38/Homo_sapiens.GRCh38.cds.all.fa.gz", description=False, func_header=lambda id_transcript:id_transcript.split(".")[0])

Reading sequence file: ./grch38/Homo_sapiens.GRCh38.cds.all.fa.gz: 109095it [00:05, 19148.90it/s]


In [11]:
sy.write_fasta(Se_cds, "./grch38/Homo_sapiens.GRCh38.cds.all.no-description.fa.gz")

In [19]:
Se_premrna.map(len).compress(lambda x:x > 10000).sort_values().head()

ENST00000441558    10001
ENST00000513749    10001
ENST00000548721    10001
ENST00000523248    10001
ENST00000575320    10001
dtype: int64

In [20]:
Se_premrna["ENST00000441558"]

'GGTCCACCCGCGAATCCTCTACAGAGATGGCCAAGCGCTGGGGTCGTGGCTGCGTCCTCTAGGCCAGgtgagaatttacaacccaccctcggccactgttgtgctgtggccgctgatgattaagcatgaggaactcagaaactctgaaccttctgacaaacgggttctcccctgcccaccgcggtgccccaaacccagcaggacagattggtttacatggtggggcattggggaggacccagagctttcaataagggaagaaatgatgtggggagggacagtaaatcaaagtaggggattaggccgggcgctgtggctcacgtctgtaatcccagccctttgggaggctgaggcagatggatctctcgagactgggagttctagaccagcctggccaacatggcgaaaccccatctctactaaaaatacgaaaattagccgggcgtggtggcgcgcgcctgtaattccagctactagagaggctgaggcaggagaatcgcttgaaccctggaggcagaggttgcaatgaggcaagatggcgccactgcactccagcctgggctacagagcgagactgtctcaaaaaaaaaaaaaaaaaaaagtaggggattagagctggccacagcttccccagtaacctgcagttactggagcaggaaggggcctcggagaatccctggaagcatctcaacagaatccagaggggacaagaaaagaggagggatacgtctgtcgcatccttcccttctcattgatccaagtttgcactagtggtgtgagccctccttatcttccaggccttggtgggcagctgttgacatgtccagaaccccaggaagtgctgaaggtacctgcagttcaggccccatggaagcccagacctccttcacccttcggggacactgaaattggccaaatgtttaggccatgaggatggctgctgagtaagcccataatccctacatcggacacaagtaagcaggtgaaggcctacaggta